In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

# Notation

This notebook solves $\frac{\mathrm{d}^2 x}{\mathrm{d}t^2} = F$, for constant $F$.

This equation does, of course, have a 2D space of solutions. This notebook finds the solution that minimizes the $L^2$ norm between the solution and a set of datapoints.

# Parameters

In [ ]:
# You can edit anything in this cell.
# If you do so, you should re-run the whole notebook.

h = 0.01  # Grid resolution
rng_seed_data = 1234  # Used for data generation
sigma = 0.0  # Noise level for data generation
N_samples = 10  # Number of datapoints to 'measure' from the 'true' curve
F = -10  # Force. Constant in this version. Chosen to be close to free-fall in SI units.

x0 = 0.5  # Initial condition for the true solution.
v0 = 0.1  # Initial condition for the true solution.

# Data generation

In [ ]:
t_grid = np.arange(0, 1, h)  # Useful to have this for plotting.
N_grid = t_grid.shape[0]  # Number of points in this discretized grid over t.
x_true_grid = x0 + t_grid*v0 + 0.5*F*t_grid**2  # True solution curve.

rng_data = np.random.RandomState(rng_seed_data)  # Instantiate the RNG in the same cell that we will do all the calls.
idx_samples = rng_data.choice(N_grid, size=N_samples, replace=False)  # Choose which datapoints will will 'measure'
idx_samples = np.sort(idx_samples)
x_noise = rng_data.normal(scale=sigma, size=(N_samples,))  # Sample noise to be added to our datapoints.
t_samples = t_grid[idx_samples]  # Used only for plotting in this example
x_samples = x_true_grid[idx_samples] + x_noise  # Noisy datapoints
del rng_data  # Delete to prevent re-use of this RNG in the solution section.

# Plot the generated data
fig, ax = plt.subplots()
ax.plot(t_grid, x_true_grid, ls='-', marker='none')
ax.plot(t_samples, x_samples, ls='none', marker='o', alpha=0.7)
ax.set_xlabel('t')
ax.set_ylabel('x(t)')

plt.show()

# Numerical solution

In [ ]:
# Parameters / initialization

# Parameters in this cell are specific to the numerical solution method.
# If you change any of these, you do not need to re-run data generation.

# The lines below define a linear transformation from the natural space of the problem,
# i.e. the vector [x(t_0), x(t_1), ..., x(t_{N-1})],
# to a potentially different space [z_0, z_1, ..., z_{N-1}] in which the ODE will be solved by gradient descent.
# To solve this in the original space, set A_x2z to be the identity matrix.
A_x2z = np.zeros((N_grid, N_grid), dtype=np.float64)
A_x2z[0, 0] = 1.0  # z_0 = z(t_0)
for i in range(1, N_grid):
    # z_i = x(t_i) - x(t_{i-1}) for i > 0
    A_x2z[i, i] = 1.0
    A_x2z[i, i-1] = -1.0


N_iter = 1000  # Number of iterations of optimization.

# Initialize the solution grid.
# I don't believe there is any benefit to using random initialization, since this problem does not
# have the same requirement for symmetry-breaking that exists with the hidden neurons of a neural network.
z_solution_grid = np.zeros(N_grid, dtype=torch.float64, requires_grad=True)

# Define the optimizer.
# This problem seems to benefit from using a second-order optimizer (which LBFGS is), and I believe that
# is due to the Hessian of loss_ODE (see below for defintion) having a very large condition number.
optimizer = torch.optim.LBFGS(lr=1, history_size=10, params=[z_solution_grid])